<a href="https://colab.research.google.com/github/castagninojose/seminar_ds/blob/master/Balance05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Balance and implementation of fights in Ludus Game
here we have all the code used for modelating and balancing the fights in Ludus


imports




In [ ]:
#@title
from itertools import combinations
import numpy as np
import plotly.express as px
import random

#Main Fight Simulator

In [ ]:
#@title
def pelea(jugador1, jugador2):
    critico = 0
    extra = 0
    #print(rango)
    player1 = BaseGladiator(jugador1)
    player2 = BaseGladiator(jugador2)
    fuerza = np.array([player1.fuerza, player2.fuerza])
    armadura = np.array([player1.armadura, player2.armadura])
    destreza = np.array([player1.destreza, player2.destreza])
    vida = np.array([player1.vida, player2.vida])
    rango = np.array([player1.rango, player2.rango])
    stun = np.array([player1.stun, player2.stun])

    if (player1.rango + player2.rango == 1):
        j = np.argmin(rango)
        i = np.argmax(rango)
        extra = 1
        #print(i,j)
    else:
        i = np.random.randint(2)
        j = np.abs(i -1)

    while ((vida[0] > 0) & (vida[1] > 0)):
        tirada = np.random.randint(200)
        #print(f'la tirada es = {tirada}')
        probstun = np.random.randint(stun[i])
        if probstun/100 >= .1:
            extra = 1 
            critico = 1
        pega_o_no  = tirada  - destreza[j]
        if tirada >= 197:
            critico = 1
        if critico == 1:
            tirada_2 = np.random.randint(100) + fuerza[i] - armadura[j]
            maximo = np.maximum(tirada_2, 40)

            vida[j] = vida[j] - maximo       
        elif pega_o_no > 0:
            tirada_2 = np.random.randint(50) + fuerza[i] - armadura[j]
            maximo = np.maximum(tirada_2, 20)

            vida[j] = vida[j] - maximo
        if extra == 1:
            extra = 0
        else:
            aux = i
            i = j
            j = aux
        critico = 0
        extra = 0 
    return vida

#Secondary Fight Simulator
Used for testing

In [ ]:
#@title
def pelea2(jugador1, jugador2, bonus1, bonus2, accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1 , accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2):
    '''
    Input:
        jugador1, jugador2 6d array, y seis millones de otras cosas
        
    Output:
        vida 2d array
    '''
    equipo1 = equip(accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1) 
    equipo2 = equip( accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2)
    [jugador1, jugador2] = premath(jugador1, jugador2, bonus1, bonus2, equipo1, equipo2)
    ##print(f"{jugador1}, {jugador2} ")
    [player1, player2, jugadores, fuerza, armadura, destreza, vida, rango, stun] = FightSetUp(jugador1, jugador2) 

    critico = 0
    extra = 0
    
    ##choose who goes first

    [Attacking_gladiator, Deffending_gladiator]= whogoesfirst(player1, player2, jugadores, rango)

    ##repit until one gladiator dies
    while ((vida[0] > 0) & (vida[1] > 0)):
        tirada = np.random.randint(200)
        probstun = np.random.randint(stun[Attacking_gladiator])
        if probstun/100 >= .1:
            extra = 1 
            critico = 1
            print(f'El gladiador {jugadores[Attacking_gladiator]} aturde al {jugadores[Deffending_gladiator]} dejandólo indefenso y haciéndole perder un turno en recuperarse')
        pega_o_no  = tirada  - destreza[Deffending_gladiator]
        if tirada >= 197:
            critico = 1
        if critico == 1:
            vida[Deffending_gladiator] = vida[Deffending_gladiator] - Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])       
            print(f'pega el jugador  {jugadores[Attacking_gladiator]} acesta un poderoso golpe crítico por {Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])} daño')
        elif pega_o_no > 0:
            vida[Deffending_gladiator] = vida[Deffending_gladiator] - Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator]) 
            print(f'El gladiador {jugadores[Attacking_gladiator]} acesta un golpe por {Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])} daño')
            #print(random_text(Attacking_gladiator,Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])))
        if extra == 0:
            aux = Attacking_gladiator
            Attacking_gladiator = Deffending_gladiator
            Deffending_gladiator = aux
        critico = 0
        extra = 0 
    print(f' El gladiador {jugadores[np.argmax(vida)]} sale victorioso con {np.max(vida)} puntos de vida finales')
    return vida

In [ ]:
def random_text(jugador,ataque):
  lista=('pega el jugador {nombre} por {hit}','pega el jugador {nombre} un golpe critico por {hit}','pega el jugador {nombre} un golpe critico y lo stunea por {hit}')
  num= np.random.randint(3)
  
  string_new=lista[num].replace('{nombre}',str(jugador))
  return string_new.replace('{hit}',str(ataque))

  

In [ ]:
def Hit(critico, fuerza, armadura):
        tirada = np.random.randint(50 + 50*critico) + fuerza - armadura
        return (np.maximum(tirada, 20 + 20*critico))

In [ ]:
def whogoesfirst(player1, player2, jugadores, rango):
    if (player1.rango + player2.rango == 1):
        Deffending_gladiator = np.argmin(rango)
        Attacking_gladiator = np.argmax(rango)
        extra = 1
        print(f'El gladiador {jugadores[i]} comienza atacando dos veces gracias a su ventaja de rango')
    else:
        Attacking_gladiator = np.random.randint(2)
        Deffending_gladiator = np.abs(Attacking_gladiator -1)
    return(Attacking_gladiator, Deffending_gladiator)    

In [ ]:
def FightSetUp(jugador1, jugador2):
    player1 = BaseGladiator(jugador1)
    player2 = BaseGladiator(jugador2)
    fuerza = np.array([player1.fuerza, player2.fuerza])
    armadura = np.array([player1.armadura, player2.armadura])
    destreza = np.array([player1.destreza, player2.destreza])
    vida = np.array([player1.vida, player2.vida])
    rango = np.array([player1.rango, player2.rango])
    stun = np.array([player1.stun, player2.stun])
    jugadores = (GLADIATORS_NAMES_LIST[player1.orden], GLADIATORS_NAMES_LIST[player2.orden])
    return(player1, player2, jugadores, fuerza, armadura, destreza, vida, rango, stun)


In [ ]:
def equip(accesorio,pantalon,armadura, escudo_o_armasecundaria,arma,casco):
    Equip =np.array([[0,0, 0,  0,  0,  0,  0],[ 0,  0,  0,  0,  0,  0,  0]])
    Equip += accesorio
    Equip += pantalon
    Equip += armadura
    Equip += escudo_o_armasecundaria
    Equip += arma
    Equip += casco
    return(Equip)

In [ ]:
def complete_attack_round(jugadores, fuerza, armadura, destreza, vida, rango, stun, Attacking_gladiator, Deffending_gladiator,critico ,extra):
    tirada = np.random.randint(200)
    probstun = np.random.randint(stun[Attacking_gladiator])
    if probstun/100 >= .1:
        extra = 1 
        critico = 1
        print('stun')
    pega_o_no  = tirada  - destreza[Deffending_gladiator]
    if tirada >= 197:
        critico = 1
        print('critico')
    if critico == 1:
        vida[Deffending_gladiator] = vida[Deffending_gladiator] - Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])       
        print(f'pega el jugador {Attacking_gladiator} por {Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])}')
    elif pega_o_no > 0:
        vida[Deffending_gladiator] = vida[Deffending_gladiator] - Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator]) 
        #print(f'pega el jugador {Attacking_gladiator} por {Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])}')
        print(random_text(Attacking_gladiator,Hit(critico, fuerza[Attacking_gladiator], armadura[Deffending_gladiator])))
    if extra == 0:
        aux = Attacking_gladiator
        Attacking_gladiator = Deffending_gladiator
        Deffending_gladiator = aux
    critico = 0
    extra = 0 
    return((Attacking_gladiator, Deffending_gladiator,critico ,extra, vida))

#Premath
calculation to add bonus and equipment to both gladiators

In [ ]:
#@title
ItemNulo = np.array([0,0,0,0,0,0,0])
BonusNulo = np.array([0,0,0,0,0,0,0])
EquipoNulo = np.array([[-1,-1,-1,-1,-1,-1,-1],[0,0,0,0,0,0,0]])

def premath(player1 = GLADIATORS_DICT['Murmillo'], player2 = GLADIATORS_DICT['Scissor'], bonus1 = BonusNulo, bonus2 = BonusNulo, equipo1 = EquipoNulo, equipo2 = EquipoNulo):
    gladiator1 = player1 + bonus1
    gladiator2 = player2 + bonus2
    gladiator1 += equipo1[0,:]
    gladiator2 -= equipo1[1,:]
    gladiator2 += equipo2[0,:]
    gladiator2 -= equipo1[1,:]
    return(gladiator1, gladiator2)

#Third Fight Simulator
with dialog report

In [ ]:
def pelea3(jugador1, jugador2, bonus1, bonus2, accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1 , accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2):
    '''
    Input:
        jugador1, jugador2 6d array, y seis millones de otras cosas
        
    Output:
        vida 2d array
    '''
    equipo1 = equip(accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1) 
    equipo2 = equip( accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2)
    [jugador1, jugador2] = premath(jugador1, jugador2, bonus1, bonus2, equipo1, equipo2)
    print(f"{jugador1}, {jugador2} ")
    [player1, player2, jugadores, fuerza, armadura, destreza, vida, rango, stun] = FightSetUp(jugador1, jugador2) 

    critico = 0
    extra = 0
    
    ##choose who goes first

    [Attacking_gladiator, Deffending_gladiator]= whogoesfirst(player1, player2, jugadores, rango)

    ##repit until one gladiator dies
    while ((vida[0] > 0) & (vida[1] > 0)):
        [Attacking_gladiator, Deffending_gladiator,critico ,extra, vida] = complete_attack_round(jugadores, fuerza, armadura, destreza, vida, rango, stun, Attacking_gladiator, Deffending_gladiator,critico ,extra)
    return vida

In [ ]:
pelea2(GLADIATORS_DICT['Cestus'],GLADIATORS_DICT['Tracio'],
    BonusNulo,
    BonusNulo,   
    Items["Accesorio"]["Anillo Emperador"]['Stats'],
    Items["Pantalon"]["Botas acero"]['Stats'], 
    Items["Armadura"]["Armadura completa"]['Stats'],
    Items["Escudo"]["Scutum"]['Stats'], 
    Items["Arma"]["Gladius"]['Stats'], 
    Items["Casco"]["Casco normal"]['Stats'],
    Items["Accesorio"]["Anillo Emperador"]['Stats'],
    Items["Pantalon"]["Botas acero"]['Stats'], 
    Items["Armadura"]["Armadura completa"]['Stats'],
    Items["Escudo"]["Scutum"]['Stats'], 
    Items["Arma"]["Gladius"]['Stats'], 
    Items["Casco"]["Casco normal"]['Stats'])

El gladiador Cestus acesta un golpe por 51 daño
El gladiador Tracio acesta un golpe por 60 daño
El gladiador Cestus acesta un golpe por 20 daño
El gladiador Tracio acesta un golpe por 57 daño
 El gladiador Tracio sale victorioso con 31 puntos de vida finales


array([-13,  31])

In [ ]:
Items["Casco"]["Casco normal"]['Stats']

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

#Teamfight 2v2




In [ ]:
def tf_2v2(jugador1, jugador2,jugador3, jugador4,
           bonus1, bonus2, bonus3, bonus4,
           accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1 , 
           accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2,
           accesorio3, pantalon3 ,armadura3, escudo_o_armasecundaria3 ,arma3 ,casco3,
           accesorio4 ,pantalon4 ,armadura4, escudo_o_armasecundaria4 ,arma4 ,casco4 
           ):
    vidaresultantepelea12 = pelea2(jugador1, jugador2, bonus1, bonus2, accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1 , accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2)
    vidaresultantepelea34 = pelea2(jugador3, jugador4, bonus3, bonus4, accesorio3 ,pantalon3 ,armadura3, escudo_o_armasecundaria3 ,arma3 ,casco3 , accesorio4 ,pantalon4 ,armadura4, escudo_o_armasecundaria4 ,arma4 ,casco4)
    jugador1[3] = vidaresultantepelea12[0]
    jugador2[3] = vidaresultantepelea12[1]
    jugador3[3]= vidaresultantepelea34[0]
    jugador4[3] = vidaresultantepelea34[1]

    if (jugador1[3] > 0) & (jugador3[3] > 0):
        print(f"gana el jugador 1 pues sus gladiadores salieron victoriosos en ambos combates")
    elif(jugador1[3] > 0):
       vidaresultantepeleaFINAL = pelea2(jugador1, jugador4, bonus1, bonus4, accesorio1 ,pantalon1 ,armadura1, escudo_o_armasecundaria1 ,arma1 ,casco1 , accesorio4 ,pantalon4 ,armadura4, escudo_o_armasecundaria4 ,arma4 ,casco4)
    elif(jugador3[3] > 0):
        vidaresultantepeleaFINAL = pelea2(jugador2, jugador3, bonus2, bonus3, accesorio2 ,pantalon2 ,armadura2, escudo_o_armasecundaria2 ,arma2 ,casco2 , accesorio3 ,pantalon3 ,armadura3, escudo_o_armasecundaria3 ,arma3 ,casco3)
    else:
        print(f"gana el jugador 2 pues sus gladiadores salieron victoriosos en ambos combates")
    return(vidaresultantepeleaFINAL)

In [ ]:
BonusNulo = np.array([0,0,0,0,0,0,0])
tf_2v2(GLADIATORS_DICT['Murmillo'],
       GLADIATORS_DICT['Tracio'],
       GLADIATORS_DICT['Provocator'],
       GLADIATORS_DICT['Scissor'],
       BonusNulo,
       BonusNulo,
       BonusNulo,
       BonusNulo, 
    Items["Accesorio"]["Anillo Emperador"]['Stats'],
    Items["Pantalon"]["Botas acero"]['Stats'], 
    Items["Armadura"]["Armadura completa"]['Stats'],
    Items["Escudo"]["Scutum"]['Stats'], 
    Items["Arma"]["Gladius"]['Stats'], 
    Items["Casco"]["Casco normal"]['Stats'],
    Items["Accesorio"]["Anillo Emperador"]['Stats'],
    Items["Pantalon"]["Botas acero"]['Stats'], 
    Items["Armadura"]["Armadura completa"]['Stats'],
    Items["Escudo"]["Scutum"]['Stats'], 
    Items["Arma"]["Gladius"]['Stats'], 
    Items["Casco"]["Casco normal"]['Stats'],
    Items["Accesorio"]["Anillo Emperador"]['Stats'],
    Items["Pantalon"]["Botas acero"]['Stats'], 
    Items["Armadura"]["Armadura completa"]['Stats'],
    Items["Escudo"]["Scutum"]['Stats'], 
    Items["Arma"]["Gladius"]['Stats'], 
    Items["Casco"]["Casco normal"]['Stats'],
    Items["Accesorio"]["Anillo Emperador"]['Stats'],
    Items["Pantalon"]["Botas acero"]['Stats'], 
    Items["Armadura"]["Armadura completa"]['Stats'],
    Items["Escudo"]["Scutum"]['Stats'], 
    Items["Arma"]["Gladius"]['Stats'], 
    Items["Casco"]["Casco normal"]['Stats'])

 El gladiador Tracio sale victorioso con 73 puntos de vida finales
 El gladiador Provocator sale victorioso con 31 puntos de vida finales
El gladiador Provocator acesta un golpe por 40 daño
El gladiador Tracio acesta un golpe por 54 daño
El gladiador Provocator acesta un golpe por 33 daño
 El gladiador Provocator sale victorioso con 12 puntos de vida finales


array([-4, 12])

#Class and dictionaries implemented
class BaseGladiator\
class FIghtersTuple\
GLADIATORS_DICT\
WEAPONS_DICT

In [ ]:
# fuerza, armadura, destreza, vida, rango, stun #
class BaseGladiator:
    def __init__(self, atributos):
        self.fuerza = atributos[0]
        self.armadura = atributos[1]
        self.destreza = atributos[2]
        self.vida = atributos[3]
        self.rango = atributos[4]
        self.stun = atributos[5]
        self.orden = atributos[6]


class FightersTuple:
    def __init__(self, player1, player2):
        self.fuerza = np.array([player1.fuerza, player2.fuerza])
        self.armadura = np.array([player1.armadura, player2.armadura])
        self.destreza = np.array([player1.destreza, player2.destreza])
        self.vida = np.array([player1.vida, player2.vida])
        self.rango = np.array([player1.rango, player2.rango])
        self.stun = np.array([player1.stun, player2.stun])

GLADIATORS_DICT = dict(
Scissor = np.array([ 85  , 37 ,  64 , 114 ,  0 ,  2  , 0]),
Murmillo = np.array([ 53 , 67 , 20  , 160 ,  0 ,  2 ,  1]),
Tracio = np.array(np.array([ 82 , 37 , 48 ,133  , 0 ,  2  , 2])),
Hoplomachis = np.array([53, 44, 80, 113, 1, 2,3]),
Provocator = np.array([ 59 , 62 , 49 , 130 ,  0 ,  2 ,  4]),
Reciario = np.array([10, 30, 55, 110,  1, 15,5]),
Crupellarius = np.array([45, 75, 35, 145,  0, 2,6]),
Cestus = np.array([ 46 , 49 , 50, 100 ,  0 , 15 ,  7]),
Lauquerius = np.array([72, 36, 48, 89,  0, 15,  8]),
Secutor = np.array([ 53 , 66 , 40 , 141 ,  0 ,  2  , 9]),
)

GLADIATORS_NAMES_LIST = ('Scissor' ,
'Murmillo',
'Tracio',
'Hoplomachis',
'Provocator',
'Reciario',
'Crupellarius',
'Cestus',
'Lauquerius',
'Secutor',)
##stun vale 55, rango vale 10 rango y stun vale 95

In [ ]:
Items = {'Accesorio': {'Amuleto de Lauquerius': {'ID': '001',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Excusivo':'Lauquerius'},
                             
                            'Amuleto en forma de espada': {'ID':'002',
                                    'Stats': np.array([[1,0,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                            'Anillo Emperador': {'ID': '003',
                                    'Stats': np.array([[-1,-1,3,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                            'Aumeleto con escudos y 2 espadas': {'ID': '004',
                                    'Stats': np.array([[1,1,-1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                            'Anillo romano': {'ID': '005',
                                    'Stats': np.array([[-1,0,-1,3,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},

                            'Aumelto lunar': {'ID': '006',
                                    'Stats': np.array([[0,-1,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                            'Brasalete romano': {'ID': '007',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                            'Cinturon de Tracio': {'ID': '007',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Tracio'},
                             
                            'Colgante de casco de Murmillo': {'ID': '007',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Murmillo'},
                             
                            'Grebas': {'ID': '008',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                            'Ocrea': {'ID': '009',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                             
                             
                             'Colgante gladiador victorioso': {'ID': '010',
                                    'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'}
                            },
         'Arma':{ 'Caestus': {'ID':'001',
                             'Stats':np.array([[2,0,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'Cestus'},
                 'Cinqueada': {'ID':'002',
                             'Stats':np.array([[1,0,1,-1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Dory lanza': {'ID':'003',
                             'Stats':np.array([[2,0,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'Hoplomachis'},
                 'Fascina tridente':{'ID':'004',
                             'Stats':np.array([[1,1,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'Reciario'},
                 'Gladius': {'ID':'004',
                             'Stats':np.array([[1,0,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Hacha doble filo': {'ID':'005',
                             'Stats':np.array([[2,-1,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Hachas de guerra': {'ID':'006',
                             'Stats':np.array([[3,-1,-1,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Latigo': {'ID':'007',
                             'Stats':np.array([[2,0,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'Laquearius'},
                 'Martillo de guerra': {'ID':'008',
                             'Stats':np.array([[3,-2,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Morning star': {'ID':'009',
                             'Stats':np.array([[1,0,1,-1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Morning star cadena': {'ID':'010',
                             'Stats':np.array([[2,-1,2,-2,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Pugio': {'ID':'011',
                             'Stats':np.array([[1,-1,3,-2,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Red': {'ID':'012',
                             'Stats':np.array([[0,0,1,1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'Reciario'},
                 'Sica': {'ID':'013',
                             'Stats':np.array([[1,-1,0,0,0,0,0],[0,1,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Spatha': {'ID':'014',
                             'Stats':np.array([[2,0,-1,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                 'Tijeras': {'ID':'015',
                             'Stats':np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0]]),
                             'Exclusivo': 'Scissor'},
                 },
         'Armadura':{'Armadura completa': {'ID':'001',
                             'Stats':np.array([[-1,3,-2,1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Armadura de Tauro': {'ID':'002',
                             'Stats':np.array([[1,2,-2,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Cardiophylax': {'ID':'003',
                             'Stats':np.array([[1,1,0,-1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Destiny leather': {'ID':'004',
                             'Stats':np.array([[-1,0,3,-1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Laminar armour': {'ID':'005',
                             'Stats':np.array([[-1,1,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Lorica Hamata': {'ID':'006',
                             'Stats':np.array([[-2,1,1,1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Lorica segmentada': {'ID':'007',
                             'Stats':np.array([[-1,-1,-1,4,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Lorica solaris': {'ID':'008',
                             'Stats':np.array([[-1,1,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Lorica squamata': {'ID':'009',
                             'Stats':np.array([[-1,1,-1,2,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     'Manica': {'ID':'010',
                             'Stats':np.array([[0,1,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'Provocator'},
                     'Tunica': {'ID':'011',
                             'Stats':np.array([[0,-1,3,-1,0,0,0],[0,0,0,0,0,0,0]]),
                             'Exclusivo': 'None'},
                     },
        'Casco':{'Casco Cestus': {'ID': '001',
                            'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                            'Excusivo':'Cestus'},

                    'Casco Hoplomacus': {'ID':'002',
                            'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                            'Exclusivo': 'Hoplomachi'},

                    'Casco Scissor': {'ID': '003',
                            'Stats': np.array([[0,0,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                            'Exclusivo': 'Scissor'},

                    'Casco Secutor': {'ID': '004',
                            'Stats': np.array([[0,1,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                            'Exclusivo': 'Secutor'},

                    'Casco normal': {'ID': '005',
                            'Stats': np.array([[0,1,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                            'Exclusivo': 'None'},
                    'Casco Plumas ': {'ID': '006',
                                    'Stats': np.array([[1,0,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Provocator'},
                    'Casco skyrim': {'ID': '007',
                                    'Stats': np.array([[-1,1,-1,2,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                    'Cassis Crista Murmillo': {'ID': '008',
                                    'Stats': np.array([[0,2,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Murmillo'} ,  
                    'Cassis Crista Tracio': {'ID': '009',
                                    'Stats': np.array([[0,0,1,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Tracio'} ,  
                    'Galea con cresta ': {'ID': '010',
                                    'Stats': np.array([[0,0,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,  
                    'Galea sin cresta ': {'ID': '011',
                                    'Stats': np.array([[0,0,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,
                    'Great Helmet': {'ID': '012',
                                    'Stats': np.array([[0,0,0,2,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Crupellarius'} ,                    
       },              
        'Escudo':{'Cetratus': {'ID': '001',
                                    'Stats': np.array([[0,1,-1,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                    'Escudo contundente': {'ID': '002',
                                    'Stats': np.array([[0,0,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                    'Escudo de Sol': {'ID': '003',
                                    'Stats': np.array([[0,1,-1,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,  
                    'Lucero': {'ID': '004',
                                    'Stats': np.array([[0,0,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,  
                    'Parma': {'ID': '005',
                                    'Stats': np.array([[-1,1,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,  
                    'Scutum': {'ID': '006',
                                    'Stats': np.array([[-1,3,-1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,
                    'looyar escudo': {'ID': '007',
                                    'Stats': np.array([[0,1,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,                    
       },         
        'Pantalon':{'Botas acero': {'ID': '001',
                                    'Stats': np.array([[1,-1,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                    'Botas largas metal': {'ID': '002',
                                    'Stats': np.array([[1,1,-2,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'},
                    'Loincloth': {'ID': '003',
                                    'Stats': np.array([[0,0,1,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,  
                    'Pantalon completo': {'ID': '004',
                                    'Stats': np.array([[0,2,0,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Crupellarius'} ,  
                    'Sandalias romanas': {'ID': '005',
                                    'Stats': np.array([[0,-1,2,0,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,  
                    'Tobillera Leon': {'ID': '006',
                                    'Stats': np.array([[0,1,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'Secutor'} ,
                    'Balteus cinturon': {'ID': '007',
                                    'Stats': np.array([[0,0,0,1,0,0,0],[0,0,0,0,0,0,0]]),
                                    'Exclusivo': 'None'} ,                    
       }

               }


sample equip

In [ ]:
equipo_Scissor = np.array(WEAPONS_DICT['Grebas_accesorio_tier1']) + np.array(WEAPONS_DICT['Loincloth_accesorio_tier1'])+ np.array(WEAPONS_DICT['Tijeras_arma_scissor_tier1']) + np.array(WEAPONS_DICT['Gladius_arma_tier1']) + np.array(WEAPONS_DICT['Casco_Scissor_tier1']) + np.array(WEAPONS_DICT['LaminarArmou_armadura_tier1'])
equipo_Tracio = np.array(WEAPONS_DICT['Grebas_accesorio_tier1']) + np.array(WEAPONS_DICT['Loincloth_accesorio_tier1'])+ np.array(WEAPONS_DICT['Scutum_escudo_tier1']) + np.array(WEAPONS_DICT['Gladius_arma_tier1']) + np.array(WEAPONS_DICT['CasisCrista_casco_Tracio_tier1']) + np.array(WEAPONS_DICT['LaminarArmou_armadura_tier1'])
print(equipo_Scissor,equipo_Tracio )

[[1 0 3 1 0 0 0]
 [0 1 0 0 0 0 0]] [[-1  5  1  1  0  0  0]
 [ 0  0  0  0  0  0  0]]


#Random Bonus Asignment

In [ ]:
#@title
#def Bonus(*arg):
bonus1 = np.array([0,0,0,0,0,0,0])
bonus2 = np.array([0,0,0,0,0,0,0])
i = np.random.randint(5)
i2 = np.random.randint(5)
for j in range(i):
    k = np.random.randint(4)
    bonus1[k] += 1
for j in range(i2):
    k = np.random.randint(4)
    bonus2[k] += 1

#    return(bonus)
print(bonus1, bonus2)

[0 1 1 0 0 0 0] [1 0 2 1 0 0 0]


In [ ]:
def levelup(item, boolean):
    bonus = np.array([0,0,0,0,0,0,0])
    if boolean == 0:
        k = np.random.randint(4)
    else:
        k = np.argmax(item['Stats'])
    bonus[k] += 1
    return(bonus)

In [ ]:
bonusej = levelup(Items["Arma"]["Red"],1)
print(bonusej)

[0 0 1 0 0 0 0]


#Examples, test and samples


In [ ]:
equipo_Scissor = np.array([[4, 0 ,0 ,0, 0, 0 ,0],
 [0, 2, 0, 0, 0, 0 ,0]])
equipo_Murmillo = np.array([[1, 4, 0,1, 0, 0 ,0],
 [0, 0,0, 0, 0, 0 ,0]])

In [ ]:
TEst = premath(GLADIATORS_DICT['Scissor'], GLADIATORS_DICT['Murmillo'], BonusNulo,BonusNulo,equipo_Scissor , equipo_Murmillo)
print(TEst)

(array([ 89,  37,  64, 114,   0,   2,   0]), array([ 54,  67,  20, 161,   0,   2,   1]))


In [ ]:
simulacion(TEst[0], TEst[1],10000)

array([5324, 4676])

In [ ]:
simulacion(GLADIATORS_DICT['Scissor'],GLADIATORS_DICT['Murmillo'], 10000)

In [ ]:
pelea3(TEst[0], TEst[1])

El gladiador Murmillo acesta un golpe por 63 daño
El gladiador Scissor acesta un golpe por 58 daño
El gladiador Murmillo acesta un golpe por 50 daño
El gladiador Scissor acesta un golpe por 23 daño
El gladiador Murmillo acesta un golpe por 30 daño


array([-29,  80])

In [ ]:
daga_index = {014, 024 }
sword_index = {017, 026, 027, 028}
axes_idnex ={018, 019}
hammer_index = {21}
morningstar_index = {22, 23}

In [ ]:
Items["Accesorio"]["Anillo Emperador"]["Stats"]

array([[-1, -1,  3,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [ ]:
equipito = equip(Items["Accesorio"]["Anillo Emperador"],
                Items["Pantalon"]["Botas acero"], 
                Items["Armadura"]["Armadura completa"],
                Items["Escudo"]["Scutum"], 
                Items["Arma"]["Gladius"], 
                Items["Casco"]["Casco normal"]
                )
print(equipito)

[[-1  5  1  1  0  0  0]
 [ 0  0  0  0  0  0  0]]


In [ ]:
for (gladiador) in GLADIATORS_DICT.keys():
    player1 = BaseGladiator(GLADIATORS_DICT[gladiador])
    print(f'Stats de {gladiador}:')
    print(player1.__dict__)

Stats de Scissor:
{'fuerza': 85, 'armadura': 37, 'destreza': 64, 'vida': 114, 'rango': 0, 'stun': 2, 'orden': 0}
Stats de Murmillo:
{'fuerza': 53, 'armadura': 67, 'destreza': 20, 'vida': 160, 'rango': 0, 'stun': 2, 'orden': 1}
Stats de Tracio:
{'fuerza': 82, 'armadura': 37, 'destreza': 48, 'vida': 133, 'rango': 0, 'stun': 2, 'orden': 2}
Stats de Hoplomachis:
{'fuerza': 53, 'armadura': 44, 'destreza': 80, 'vida': 113, 'rango': 1, 'stun': 2, 'orden': 3}
Stats de Provocator:
{'fuerza': 59, 'armadura': 62, 'destreza': 49, 'vida': 130, 'rango': 0, 'stun': 2, 'orden': 4}
Stats de Reciario:
{'fuerza': 10, 'armadura': 30, 'destreza': 55, 'vida': 110, 'rango': 1, 'stun': 15, 'orden': 5}
Stats de Crupellarius:
{'fuerza': 45, 'armadura': 75, 'destreza': 35, 'vida': 145, 'rango': 0, 'stun': 2, 'orden': 6}
Stats de Cestus:
{'fuerza': 46, 'armadura': 49, 'destreza': 50, 'vida': 100, 'rango': 0, 'stun': 15, 'orden': 7}
Stats de Lauquerius:
{'fuerza': 72, 'armadura': 36, 'destreza': 48, 'vida': 89, 'r

#simulacion function
used to run $n$ fights between $player1$ and $player2$

In [ ]:
#@title
def simulacion(player1, player2, n):
  rv = np.array([0, 0])
  for k in range(n):
        round = pelea(player1, player2)
        #print(rv)
        if round[0] > round[1]:
            rv[0] += 1
        else:
            rv[1] += 1
  return(rv)
    


#Big all-vs-all simulation

In [ ]:
A = np.zeros((10,10))
B = np.zeros((10,10))

for (gladiador1, gladiador2) in combinations(GLADIATORS_DICT.keys(), 2):
    simul = simulacion(GLADIATORS_DICT[gladiador1], GLADIATORS_DICT[gladiador2], 1000000)
    player1 = BaseGladiator(GLADIATORS_DICT[gladiador1])
    player2 = BaseGladiator(GLADIATORS_DICT[gladiador2])
    print(f' {gladiador1} {simul[0]} vs {gladiador2} {simul[1]}')
    A[player1.orden, player2.orden] = simul[0]
    A[player2.orden, player1.orden] = simul[1]

print(A)

for i in range(10):
    for j in range(10):
        if (A[i,j] > 530000) & (i != j) :
           B[i,j] = 1
        elif (A[i,j] < 470000) & (i != j):
            B[i,j] = -1

b=np.sum(B,axis=1)
a=np.sum(A,axis=1)

print(f'el vector suma es{a}')
print(B)
print(f'el vector suma es{b}')

NameError: ignored

#Pie Plot
Pie plot of 1kk simulation of every gladiator without equip or bonus

Actual


In [ ]:
#este es el actual!
import plotly.graph_objects as go

labels = ['Scissor', 'Murmillo', 'Tracio', 'Hoplomachis', 'Provocator', 'Reciario', 'Crupellarius', 'Cestus', 'Lauquerius', 'Secutor']
values = [4514708, 4614288, 4514906, 4402654, 4562032, 4533075, 4412468, 4575796, 4421672, 4448401]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.show()

#Semi-auto balance Heuristic

In [ ]:
#@title
index = {0,1,2,3}
p1 = 0
p2 = 0
for (i, j) in combinations(index, 2):
    for k in range(4):
        k = k+1
        if ((GLADIATORS_DICT['Provocator'][j] - k) > 0) & ((GLADIATORS_DICT['Provocator'][i] - k) > 0):
            aux1 =  np.array([ 63 , 58 , 49 ,130 ,  0 ,  2  , 4])
            aux1[i] -= k
            aux1[j] += k
            aux2 =   np.array([ 63 , 58 , 49 ,130 ,  0 ,  2  , 4])
            aux2[i] += k
            aux2[j] -= k 
            for gladiador2 in GLADIATORS_DICT.keys():
                simul1 = simulacion(aux1, GLADIATORS_DICT[gladiador2],1000000)
                simul2 = simulacion(aux2, GLADIATORS_DICT[gladiador2],1000000)
                if simul1[0] > 530000:
                    p1 = p1 + 1
                elif simul1[0] < 470000:
                    p1 = p1 -1
                if simul2[0] > 530000:
                    p2 = p2 + 1
                elif simul2[0] < 470000:
                    p2 = p2 -1
            print(f'p1 es:{p1} con {aux1}')
            print(f'p2 es:{p2} con {aux2}')
            p1 = 0
            p2 = 0       

p1 es:1 con [ 62  59  49 130   0   2   4]
p2 es:2 con [ 64  57  49 130   0   2   4]
p1 es:1 con [ 61  60  49 130   0   2   4]
p2 es:2 con [ 65  56  49 130   0   2   4]
p1 es:0 con [ 60  61  49 130   0   2   4]
p2 es:2 con [ 66  55  49 130   0   2   4]
p1 es:0 con [ 59  62  49 130   0   2   4]
p2 es:2 con [ 67  54  49 130   0   2   4]
p1 es:1 con [ 62  58  50 130   0   2   4]
p2 es:3 con [ 64  58  48 130   0   2   4]


In [ ]:
horarios = list(range(0, 23))
primetime = list(range(0,3))
distribucionfinal = []
horario1 = random.choice(primetime)
distribucionfinal.append(horario1)
horarios.remove(horario1)
primetime.remove(horario1)
horario2 = random.choice(primetime)
distribucionfinal.append(horario2)
horarios.remove(horario2)
primetime.remove(horario2)

while len(horarios) > 1:
    nuevohorario = random.choice(horarios)
    distribucionfinal.append(nuevohorario)
    horarios.remove(nuevohorario)
    if horarios.count(nuevohorario + 1) > 0:
        horarios.remove(nuevohorario + 1)
        i = 1
    elif horarios.count(nuevohorario - 1) > 0:
        horarios.remove(nuevohorario - 1)
        j = 1
    nuevohorario2 = random.choice(horarios)
    distribucionfinal.append(nuevohorario2)
    horarios.remove(nuevohorario2)
    if i == 1:
        horarios.append(nuevohorario + 1)
        i = 0
    elif j == 1:
        horarios.append(nuevohorario - 1)
        j = 0
    
print(distribucionfinal)


[1, 0, 12, 17, 11, 2, 6, 8, 5, 14, 3, 18, 16, 22, 15, 13, 9, 19, 4, 20, 7, 21]
